# API Exercises

Here are two exercises that involve working with APIs and dictionaries.

One is using the Open Brewery API found at https://www.openbrewerydb.org/, and the other is using the API for UK Police Data, found at https://data.police.uk/docs/.

You can complete them in either order!

Remember that you can create new cells with esc + a or b

## Breweries

### Q1: Load the first page of results with 50 results per page

In [373]:
import json
import requests
url = 'https://api.openbrewerydb.org/breweries?page=1&per_page=50'
url_api = requests.get(url).json()
url_api;

### Q2: This is only the first 50 results.  Get the next 50 and put them together.

In [374]:
import json
import requests
url_nextpage = 'https://api.openbrewerydb.org/breweries?page=2&per_page=100' #change page 1 to 2
url_api2 = requests.get(url_nextpage).json()
brewery_list = url_api + url_api2
brewery_list;

In [293]:
brewery_list[0]['name']

'Avondale Brewing Co'

### Q3: How many of these 100 breweries in are in Alaska?

In [294]:
len(list(filter(lambda brewery: brewery['state'] == 'Alaska', brewery_list)))

3

### Q4: Of these 100 breweries, what are the different unique brewery types?

In [295]:
set(list(map(lambda brewery: brewery['brewery_type'], brewery_list)))


{'brewpub', 'contract', 'micro', 'planning', 'proprietor', 'regional'}

### Q5: What is the closest brewery to "Devil's Potion Brewing Company LLC" ?
* Hint 1: Use Euclidian distance w/ longitude and latitude (assume longitude and latitude are a Carteisan coordinate system)
* Hint 2: You'll have to ignore the entries with "none" for latitude or longitude

In [297]:
import math

my_brewery_1 = list(filter(lambda brewery: brewery['name'] == "Devil's Potion Brewing Company LLC", brewery_list))
my_brewery = my_brewery_1.pop()
my_y = float(my_brewery['longitude'])
my_x = float(my_brewery['latitude'])

no_nones_long = list(filter(lambda brewery: brewery['longitude'] != None, brewery_list))
list_long = list(map(lambda brewery: float(brewery['longitude']), no_nones_long))

no_nones_lat = list(filter(lambda brewery: brewery['latitude'] != None, brewery_list))
list_lat = list(map(lambda brewery: float(brewery['latitude']), no_nones_lat))

locations = list(zip(list_lat, list_long))

distances = []
for i in range(0,len(locations)):
# shortest distance btw. 2 points: sqrt((x2 - x1)^2 + (y2 - y1)^2)
    new = (math.sqrt((my_x - locations[i][0])**2 + (my_y - locations[i][1])**2))
    distances.append(new)

index = distances.index(sorted(distances)[1])
closest = (list(filter(lambda brewery: brewery['latitude'] == str(locations[index][0]) \
            and brewery['longitude'] == str(locations[index][1]), brewery_list))).pop()
closest['name']

'Port Brewing Co / The Lost Abbey'

### Q6: Write a function to find the closest brewery to any other given brewery

In [298]:
def closest_brewery(brewery_here):
    import math
    
    interest = list(filter(lambda brewery: brewery['name'] == brewery_here, brewery_list)).pop()
    my_y = float(interest['longitude'])
    my_x = float(interest['latitude'])

    no_nones_long = list(filter(lambda brewery: brewery['longitude'] != None, brewery_list))
    list_long = list(map(lambda brewery: float(brewery['longitude']), no_nones_long))

    no_nones_lat = list(filter(lambda brewery: brewery['latitude'] != None, brewery_list))
    list_lat = list(map(lambda brewery: float(brewery['latitude']), no_nones_lat))

    locations = list(zip(list_lat, list_long))

    distances = []
    for i in range(0,len(locations)):
    # shortest distance btw. 2 points: sqrt((x2 - x1)^2 + (y2 - y1)^2)
        new = (math.sqrt((my_x - locations[i][0])**2 + (my_y - locations[i][1])**2))
        distances.append(new)

    index = distances.index(sorted(distances)[1])
    closest = (list(filter(lambda brewery: brewery['latitude'] == str(locations[index][0]) \
            and brewery['longitude'] == str(locations[index][1]), brewery_list))).pop()
    return closest['name']
    
    
    

In [299]:
closest_brewery("Devil's Potion Brewing Company LLC" )

'Port Brewing Co / The Lost Abbey'

### Q7: How would you get the first 10 pages from this API and put them all together using a for loop?

In [304]:
import json
import requests

page = list(range(1,11))
API = []
for i in range(0,10):
    url = 'https://api.openbrewerydb.org/breweries?page={number}&per_page=50'.format(number \
                                                                                     = page[i])
    API += requests.get(url).json() #outputs list of dictionaries
len(API)

500

# Crime in the UK

### We will be analyzing different crimes reported in the UK as provided by https://data.police.uk/docs/

# Exploratory analysis
##### 1. How many total crimes were there at latitude : 52.63902 and -1.131321 on November of 2017.
Use the street level crimes data, the documentation for the API can be found at https://data.police.uk/docs/method/crime-street/

In [305]:
import json
import requests

UK = 'https://data.police.uk/api/crimes-street/all-crime?lat=52.63902&lng=-1.131321&date=2017-11'
UK_api = requests.get(UK).json()
len(UK_api)


1334

##### 2. We've queried the API once, but it could get annoying to retype the url over and over again, create a function `make_api_request` that enables you to query the API.


 The parameters for the function should be:
* lat (float) : latitude
* lng (float) : longitude
* date (string): Date in the format YYYY-MM
    * default value = `None`
    
And it should return a json object of 

for more information on default values check out http://blog.thedigitalcatonline.com/blog/2015/02/11/default-arguments-in-python/

In [306]:
def make_api_request(lat, lng, date = None):
    if date != None:
        UK = 'https://data.police.uk/api/crimes-street/all-crime?lat={lat}&lng={lng}1&date={date}'.format(lat = lat, lng = lng, date = date)
        return requests.get(UK).json()
    else:
        UK = 'https://data.police.uk/api/crimes-street/all-crime?lat={lat}&lng={lng}1'.format(lat = lat, lng = lng)
        return requests.get(UK).json()


    
    

In [375]:
make_api_request(52.63902,-1.131321);


In [308]:
crime[0]['category']

'anti-social-behaviour'

##### 3. Write a function `categories_of_crime` that will determine the count of each type of crime for a given latitude and longitude. This is labelled as 'category' in the records. Your function should call the `make_api_request` function you created.

The parameters for the function should be:

* lat (float) : latitude
* lng (float) : longitude
* date (str) default = None

The function should return:
* a dictionary with the count of each type of crime



Once you've created the function, try it with these locations
* lat, lng of 51.5017861,-0.1432319   (Buckingham Palace)
* lat, lng of 53.480161, -2.245163     (Manchester)

In [320]:
def categories_of_crime(lat, lng, date = None):
    crimes = make_api_request(lat, lng, date = None)
    categories = (list(map(lambda crime: crime['category'], crimes)))
    crimes_unique = set(categories)
    crimes_u = dict.fromkeys(crimes_unique, 0)
    for category in categories:
        crimes_u[category] += 1
    return crimes_u


In [321]:
categories_of_crime(51.5017861,-0.1432319)

{'robbery': 141,
 'criminal-damage-arson': 82,
 'other-theft': 810,
 'violent-crime': 553,
 'possession-of-weapons': 26,
 'vehicle-crime': 102,
 'public-order': 174,
 'shoplifting': 329,
 'drugs': 74,
 'other-crime': 22,
 'bicycle-theft': 65,
 'theft-from-the-person': 586,
 'anti-social-behaviour': 608,
 'burglary': 138}

In [322]:
categories_of_crime(53.480161,-2.245163)

{'robbery': 134,
 'criminal-damage-arson': 132,
 'other-theft': 357,
 'violent-crime': 607,
 'possession-of-weapons': 25,
 'vehicle-crime': 284,
 'public-order': 289,
 'shoplifting': 159,
 'drugs': 42,
 'other-crime': 25,
 'bicycle-theft': 86,
 'theft-from-the-person': 321,
 'anti-social-behaviour': 422,
 'burglary': 95}

**Bonus**: 
* Write a function that determines the difference between Buckingham Palace and Manchester in terms of the number of crimes in each category.
    * In which category is there the largest absolute difference between the category of crime?
* Create a histogram depiction of the categories of crime

In [369]:
def most_crimes():
    manchester_crimes_dict = categories_of_crime(53.480161,-2.245163) 
    buckingham_crimes_dict = categories_of_crime(51.5017861,-0.1432319)
    global crime_list
    crime_list = list(manchester_crimes_dict.keys())
    m = list(manchester_crimes_dict.values())
    b = list(buckingham_crimes_dict.values())
    zipped = list(zip(m,b))
    counts = list(map(lambda tup: tup[0] - tup[1],zipped))
    global abs_count
    abs_count = list(map(lambda count: abs(count), counts))
    return dict(zip(crime_list,abs_count))
most_crimes()

{'robbery': 7,
 'criminal-damage-arson': 50,
 'other-theft': 453,
 'violent-crime': 54,
 'possession-of-weapons': 1,
 'vehicle-crime': 182,
 'public-order': 115,
 'shoplifting': 170,
 'drugs': 32,
 'other-crime': 3,
 'bicycle-theft': 21,
 'theft-from-the-person': 265,
 'anti-social-behaviour': 186,
 'burglary': 43}

In [371]:
!pip install plotly
import plotly.graph_objs

In [372]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

trace = {'type': 'bar', 'x' : crime_list, 'y' : abs_count}
layout = {'title' : 'Difference btw. Manchester and Buckingham Palace Crime Category Count'}

figure = {'data':[trace], 'layout':layout}
plotly.offline.iplot(figure)

##### 4. Create a function `find_outcome_statuses` that will determine outcome statuses for a given latitude and longitude and date (optional)?
Investigate the data to determine where the outcome statuses are located.

**NOTE**: You'll notice that some of these crimes do not have crime outcomes. Make these into the category of "Not Resolved."

**NOTE 2**: These might take a long time to execute if you do not specify a month

**Bonus**: What is the ratio of crimes investigated to those not investigated? Is it higher near London or Manchester?

##### 5. Write a function `month_highest_crimes` that will return the month that had the highest number of crimes for a latitude, longitude and a year.

Inputs
* lat (float) : latitude
* lng (float) : longitude
* year (str) : in the format YYYY

Output
* month with highest crime (int)

**Bonus** Make a graph of how the number of crimes changed over time for a year. This will likely require a new function. Is seasonality a factor? Do the type of crimes change over time?

### Bonus Open Ended Questions

1. Take a look at the https://data.police.uk/docs/method/stops-street/ API. Is there a correlation between gender and being stopped and searched? How about race and being stopped and searched?